In [3]:
import arcpy
import numpy as np
import pandas as pd

In [145]:
arcpy.env.workspace = "C:\\Users\\dtian2\\Documents\\ArcGIS\\Projects\\Topkriging1\\Topkriging2.gdb"

In [69]:
# reload the table 

table = "C:\\Users\\dtian2\\Documents\\ArcGIS\\Projects\\Topkriging1\\Topkriging2.gdb\\meas"
title = ('ID','I','J','Area_i','Area_j','Meas','f_ij','f_ii','f_jj','reg_f','nugget','reg_var')
var_np = arcpy.da.TableToNumPyArray(table, title)
var = pd.DataFrame.from_records(var_np)

In [146]:
var.head(10)

,ID,I,J,Area_i,Area_j,Meas,f_ij,f_ii,f_jj,reg_f,nugget,reg_var,inf,cov
0,1,1.0,1.0,104.485128,104.485128,104.485128,691.910662,691.910662,691.910662,0.000000,-6.938894e-18,-6.938894e-18,-0.059784,-0.059784
1,2,1.0,2.0,104.485128,60.145781,0.000000,1905.633495,691.910662,3751.406041,-0.246960,9.588106e-03,-2.373717e-01,-0.543951,-0.296991
2,3,1.0,3.0,104.485128,3.994865,0.000000,3609.755386,691.910662,9232.257378,-280.633771,9.512050e-02,-2.805387e+02,-289.187870,-8.554100
3,4,1.0,4.0,104.485128,125.197986,104.486155,842.957244,691.910662,967.362279,0.001893,5.794637e-04,2.472246e-03,-0.059547,-0.061440
4,5,1.0,5.0,104.485128,0.091852,0.000000,105.810779,691.910662,4.430023,-251.550823,3.988192e+00,-2.475626e+02,-258.487358,-6.936534
5,6,1.0,6.0,104.485128,41.521457,0.000000,1217.019183,691.910662,1689.037369,-0.241016,1.231761e-02,-2.286985e-01,-0.512495,-0.271479
6,7,1.0,7.0,104.485128,18.568857,0.000000,2199.120798,691.910662,4997.747972,-6.145484,2.321331e-02,-6.122270e+00,-7.258727,-1.113243
7,8,1.0,8.0,104.485128,2.371491,0.000000,2654.307831,691.910662,3042.443172,-259.808407,1.578361e-01,-2.596506e+02,-270.362159,-10.553753
8,9,1.0,9.0,104.485128,88.475986,0.000000,4197.812259,691.910662,8231.440266,-0.103367,7.639607e-03,-9.572738e-02,-0.553214,-0.449847
9,10,1.0,10.0,104.485128,27.937138,0.000000,868.360021,691.910662,749.387099,-0.214284,1.660373e-02,-1.976802e-01,-0.498325,-0.284041


# generate the table

In [6]:
var = pd.DataFrame(np.zeros([47*47,12], dtype = float),
                   columns = ['ID','I','J','Area_i','Area_j','Meas','f_ij','f_ii','f_jj','reg_f','nugget','reg_var'])

# ID
var.loc[:,'ID'] = range(1,47*47+1)

# I & J
i = 0
a = 0
for i in range(1,48):
    j = 1
    for j in range(1,48):
        var.loc[a,'I'] = i
        var.loc[a,'J'] = j
        a += 1
        j += 1
    i += 1
    
var.head()

,ID,I,J,Area_i,Area_j,Meas,f_ij,f_ii,f_jj,reg_f,nugget,reg_var
0,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#  Area and Meas

In [7]:
# Area_i, Area_j

a = [row[0] for row in arcpy.da.SearchCursor("observation", "Area")]

for i in range(47):
    var.loc[var['I'] == i+1, 'Area_i'] = a[i]
    var.loc[var['J'] == i+1, 'Area_j'] = a[i]
    
var.head()

,ID,I,J,Area_i,Area_j,Meas,f_ij,f_ii,f_jj,reg_f,nugget,reg_var
0,1,1.0,1.0,104.485128,104.485128,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.0,2.0,104.485128,60.145781,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1.0,3.0,104.485128,3.994865,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1.0,4.0,104.485128,125.197986,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1.0,5.0,104.485128,0.091852,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Meas 
arcpy.Copy_management("observation", "observationCopy")

for i in range(47):
    for j in range(47):
        
        area1 = arcpy.management.SelectLayerByAttribute("observation", "NEW_SELECTION", "OBJECTID = {}".format(i+1)) # OID = I

        area2 = arcpy.management.SelectLayerByAttribute("observationCopy", "NEW_SELECTION", "OBJECTID = {}".format(j+1)) # OID = J

        arcpy.analysis.TabulateIntersection(area1, "OBJECTID", area2, "intersect_{}_{}".format(i+1,j+1), "OBJECTID", "Area")

        intersect = "intersect_{}_{}".format(i+1,j+1)
        a = [row[0] for row in arcpy.da.SearchCursor(intersect, "Area")]
        
        if len(a) == 0:
            var.loc[(var['I'] == i+1) & (var['J'] == j+1), 'Meas'] = 0
        else:
            var.loc[(var['I'] == i+1) & (var['J'] == j+1), 'Meas'] = a[0]

In [10]:
# save to table 

var_np = var.to_records()
table = "C:\\Users\\DTian\\Documents\\ArcGIS\\Projects\\Topkriging1\\Topkriging1.gdb\\meas"
arcpy.da.NumPyArrayToTable(var_np,table)

# Read an attribute table as data frame

In [14]:

## not useful here

in_table = "C:\\Users\\DTian\\Documents\\ArcGIS\\Projects\\Topkriging1\\Topkriging1.gdb\\observation"

# to excel

out_xls = "C:\\Users\\DTian\\Documents\\ArcGIS\\Projects\\Topkriging1\\observation.xls"
arcpy.TableToExcel_conversion(in_table, out_xls)


# import as pandas dataframe

df = pd.read_excel(out_xls)
df.head()

,OBJECTID,site_no,station_nm,dec_lat_va,dec_long_v,ListID,mu,sigma,gamma,Area,drain_area,new_area,Shape_Length,Shape_Area
0,1,8068720,"Cypress Ck at Katy-Hockley Rd nr Hockley, TX",29.950224,-95.808284,3,6.956150,1.024465,-1.164972,104.485128,110.00,104.223040,413725.460571,2.905130e+09
1,2,8072300,"Buffalo Bayou nr Katy, TX",29.743287,-95.806895,4,7.627465,0.629465,0.309767,60.145781,63.30,60.013551,323807.053637,1.672727e+09
2,3,8068300,"Mill Ck Trib nr Dobbin, TX",30.260488,-95.770782,5,5.432141,2.000901,-0.590777,3.994865,4.07,3.982606,68630.753305,1.110274e+08
3,4,8068740,"Cypress Ck at House-Hahl Rd nr Cypress, TX",29.959112,-95.717725,9,7.410441,0.843030,1.261535,125.197986,131.00,124.884240,525512.130271,3.481041e+09
4,5,8067750,"Landrum Ck Trib nr Montgomery, TX",30.351041,-95.697447,11,4.236061,0.700468,-1.422817,0.091852,0.13,0.091556,10524.700701,2.552516e+06


# Select discrete points

In [36]:
# select discrete points

for i in range(1,48):
    shed = arcpy.management.SelectLayerByAttribute("observation", "NEW_SELECTION", "OBJECTID = {}".format(i))
    
    arcpy.management.CopyFeatures(shed, "shed_{}".format(i))
    
    a = [row[0] for row in arcpy.da.SearchCursor("shed_{}".format(i), "Area")]
    area = a[0]
    
    if area >= 300:
        pnt = arcpy.management.SelectLayerByLocation("fishnet1_label", "WITHIN", "shed_{}".format(i))
    elif area >= 100:
        pnt = arcpy.management.SelectLayerByLocation("fishnet2_label", "WITHIN", "shed_{}".format(i))
    elif area >= 20:
        pnt = arcpy.management.SelectLayerByLocation("fishnet3_label", "WITHIN", "shed_{}".format(i))
    elif area >= 4:
        pnt = arcpy.management.SelectLayerByLocation("fishnet4_label", "WITHIN", "shed_{}".format(i))
    else:
        pnt = arcpy.management.SelectLayerByLocation("fishnet5_label", "WITHIN", "shed_{}".format(i))
    
    arcpy.management.CopyFeatures(pnt,"pnt_{}".format(i))

# Calculate variance

In [147]:
location = "C:\\Users\\dtian2\\Documents\\ArcGIS\\Projects\\Topkriging1\\Topkriging2.gdb"

In [149]:
#  generate near table

for i in range(1,48):
    for j in range(1,48):
        
        # generate near table
        arcpy.analysis.GenerateNearTable("pnt_{}".format(i), "pnt_{}".format(j), 
                                              "near_{}_{}".format(i,j), closest = "ALL")
        
        # calculate point variance         
        tb = "near_{}_{}".format(i,j)
        arcpy.management.AddField(tb, "f_d", "DOUBLE")
        arcpy.management.CalculateField(tb, "f_d", 
                                        "0.7320 + 1.23 * (1 - math.exp(-!NEAR_DIST!/300000))", 
                                        "PYTHON3")
        
 

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 19)

In [150]:
#  f(h) 

for i in range(1,48):
    for j in range(1,48):
        
        tb = "near_{}_{}".format(i,j)

        # calculate f_ij
        arcpy.analysis.Statistics(tb, "mean_{}_{}".format(i,j), [["f_d", "MEAN"]])
        stats = "mean_{}_{}".format(i,j)
        a = [row[0] for row in arcpy.da.SearchCursor(stats, "MEAN_f_d")]
        if len(a) == 0:
            var.loc[(var['I'] == i) & (var['J'] == j), 'f_ij'] = 0
        else:
            var.loc[(var['I'] == i) & (var['J'] == j), 'f_ij'] = a[0]
        
        # f_ii & f_jj
        if i == j:
            var.loc[var['I'] == i, 'f_ii'] = a[0]
            var.loc[var['J'] == i, 'f_jj'] = a[0]
            
        
            
# save to csv 

var.to_csv("C:\\Users\\dtian2\\Documents\\ArcGIS\\Projects\\Topkriging1\\variance",  sep='\t')

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Table: Dataset near_1_1 does not exist or is not supported
Failed to execute (Statistics).


'near_1_1'

In [105]:
# regularised semivariance

var.reg_f = var.f_ij/(var.Area_i*var.Area_j) - 0.5 * (var.f_ii/var.Area_i**2 + var.f_jj/var.Area_j**2)

var.nugget = 0.5 * (0.7320/var.Area_i + 0.7320/var.Area_j - 2*0.7320*var.Meas/(var.Area_i*var.Area_j))

var.reg_var = var.reg_f + var.nugget
var.head()

,ID,I,J,Area_i,Area_j,Meas,f_ij,f_ii,f_jj,reg_f,nugget,reg_var
0,1,1.0,1.0,104.485128,104.485128,104.485128,691.910662,691.910662,691.910662,0.000000,-6.938894e-18,-6.938894e-18
1,2,1.0,2.0,104.485128,60.145781,0.000000,1905.633495,691.910662,3751.406041,-0.246960,9.588106e-03,-2.373717e-01
2,3,1.0,3.0,104.485128,3.994865,0.000000,3609.755386,691.910662,9232.257378,-280.633771,9.512050e-02,-2.805387e+02
3,4,1.0,4.0,104.485128,125.197986,104.486155,842.957244,691.910662,967.362279,0.001893,5.794637e-04,2.472246e-03
4,5,1.0,5.0,104.485128,0.091852,0.000000,105.810779,691.910662,4.430023,-251.550823,3.988192e+00,-2.475626e+02


In [108]:
# save to table 

var_np = var.to_records()
table = "C:\\Users\\dtian2\\Documents\\ArcGIS\\Projects\\Topkriging1\\Topkriging2.gdb\\reg_var"
arcpy.da.NumPyArrayToTable(var_np,table)

In [136]:
var['inf'] = (1.962*20) / (var.Area_i*var.Area_j) - 0.5 * (var.f_ii/var.Area_i**2 + var.f_jj/var.Area_j**2)

var['cov'] = var.inf + var.nugget - var.reg_var

In [142]:
var.tail()

,ID,I,J,Area_i,Area_j,Meas,f_ij,f_ii,f_jj,reg_f,nugget,reg_var,inf,cov
2204,2205,47.0,43.0,189.60051,118.736289,0.000000,1771.113901,2538.481366,878.472325,0.012210,5.012836e-03,1.722286e-02,-0.064720,-0.076930
2205,2206,47.0,44.0,189.60051,2817.262970,189.567538,12868.206419,2538.481366,44048.857297,-0.013991,1.800506e-03,-1.219095e-02,-0.038009,-0.024017
2206,2207,47.0,45.0,189.60051,386.628019,0.000000,1671.442039,2538.481366,773.915808,-0.015095,2.877021e-03,-1.221775e-02,-0.037361,-0.022266
2207,2208,47.0,46.0,189.60051,331.856223,0.000000,1424.155814,2538.481366,483.603403,-0.014869,3.033262e-03,-1.183536e-02,-0.036879,-0.022011
2208,2209,47.0,47.0,189.60051,189.600510,189.600510,2538.481366,2538.481366,2538.481366,0.000000,-1.387779e-17,-1.387779e-17,-0.069523,-0.069523


In [139]:
# save to table 

var_np = var.to_records()
table = "C:\\Users\\dtian2\\Documents\\ArcGIS\\Projects\\Topkriging1\\Topkriging2.gdb\\covariance"
arcpy.da.NumPyArrayToTable(var_np,table)

In [143]:
# save to csv

var.to_csv("C:\\Users\\dtian2\\Documents\\ArcGIS\\Projects\\Topkriging1\\covariance",  sep='\t')

# copy to Topkriging2.gdb

In [1]:
arcpy.env.workspace = "C:\\Users\\dtian2\\Documents\\ArcGIS\\Projects\\Topkriging1\\Topkriging1.gdb"

In [7]:
location = "C:\\Users\\dtian2\\Documents\\ArcGIS\\Projects\\Topkriging1\\Tookriging3.gdb"


In [4]:
tables = arcpy.ListTables("near_*", "ALL")

In [5]:
for table in tables: 
    print(table)

near_1_2
near_1_3
near_1_4
near_1_5
near_1_6
near_1_7
near_1_8
near_1_9
near_1_10
near_1_11
near_1_12
near_1_13
near_1_14
near_1_15
near_1_16
near_1_17
near_1_18
near_1_19
near_1_20
near_1_21
near_1_22
near_1_23
near_1_24
near_1_25
near_1_26
near_1_27
near_1_28
near_1_29
near_1_30
near_1_31
near_1_32
near_1_33
near_1_34
near_1_35
near_1_36
near_1_37
near_1_38
near_1_39
near_1_40
near_1_41
near_1_42
near_1_43
near_1_44
near_1_45
near_1_46
near_2_1
near_2_2
near_2_3
near_2_4
near_2_5
near_2_6
near_2_7
near_2_8
near_2_9
near_2_10
near_2_11
near_2_12
near_2_13
near_2_14
near_2_15
near_2_16
near_2_17
near_2_18
near_2_19
near_2_20
near_2_21
near_2_22
near_2_23
near_2_24
near_2_25
near_2_26
near_2_27
near_2_28
near_2_29
near_2_30
near_2_31
near_2_32
near_2_33
near_2_34
near_2_35
near_2_36
near_2_37
near_2_38
near_2_39
near_2_40
near_2_41
near_2_42
near_2_43
near_2_44
near_2_45
near_2_46
near_3_1
near_3_2
near_3_3
near_3_4
near_3_5
near_3_6
near_3_7
near_3_8
near_3_9
near_3_10
near_3_11
near_3

In [ ]:
for table in tables: 
    # Copy the features from the workspace to a folder
    arcpy.TableToGeodatabase_conversion(table, location)
    